In [ ]:
import usb.core
import usb.util
import struct
import numpy as np
import time

# Find your device
dev = usb.core.find(idVendor=0x359F, idProduct=0x3031)

# Check if the device was found
if dev is None:
    dev = usb.core.find(idVendor=0x359F, idProduct=0x3021)
elif dev is None:
    dev = usb.core.find(idVendor=0x359F, idProduct=0x3031)
    if dev is None:
        raise ValueError('Device not found')

# Print the device speed
speed_dict = {
    1: "Low Speed",  # USB 1.0
    2: "Full Speed", # USB 1.1
    3: "High Speed", # USB 2.0
    4: "Super Speed" # USB 3.0
}

print(f'Device speed: {speed_dict.get(dev.speed, "Unknown Speed")}')

# Set the configuration
dev.set_configuration()

# Get an endpoint instance
cfg = dev.get_active_configuration()
intf = cfg[(0, 0)]

print(intf)

ep_out = usb.util.find_descriptor(
    intf,
    custom_match=lambda e: usb.util.endpoint_direction(e.bEndpointAddress) == usb.util.ENDPOINT_OUT and e.bEndpointAddress == 0x02
)

ep_in = usb.util.find_descriptor(
    intf,
    custom_match=lambda e: usb.util.endpoint_direction(e.bEndpointAddress) == usb.util.ENDPOINT_IN and e.bEndpointAddress == 0x82
)

if ep_out is None or ep_in is None:
    raise ValueError('Endpoint not found')

print('\n\n')

Voltage = 1.0

PWM_Val = int((Voltage / 3.3) * 1024)

value_bytes = struct.unpack("<2B", np.uint16(PWM_Val))
print(f'Set Voltage: {Voltage}V')
print(f'PWM 1to0 Value: {np.uint16(PWM_Val)} {[hex(x) for x in value_bytes]}')
print('\n')

bmRequestType = 0b01000000
bRequest = 1
wValue = 0x04
wIndex = 0
data = [0x00, 0x00, 0x00, 0x00]
data_hex = [hex(x) for x in data]
print('REG CTRL Write data:', data_hex)
# Perform control transfer
response = dev.ctrl_transfer(bmRequestType, bRequest, wValue, wIndex, data)
print('REG CTRL Write Bytes:', response)
print('\n')

bmRequestType = 0b01000000
bRequest = 1
wValue = 0x0C
wIndex = 0
data = [0x03, 0x00, 0x00, 0x00]
data_hex = [hex(x) for x in data]
print('REG AUX Write data:', data_hex)
# Perform control transfer
response = dev.ctrl_transfer(bmRequestType, bRequest, wValue, wIndex, data)
print('REG AUX Write Bytes::', response)
print('\n')

## VREF REG
# Define control transfer parameters
bmRequestType = 0b01000000
bRequest = 1
wValue = 0x10
wIndex = 0
data = list(value_bytes)
data_hex = [hex(x) for x in data]
print('VREF Write data:', data_hex)
# Perform control transfer
response1 = dev.ctrl_transfer(bmRequestType, bRequest, wValue, wIndex, data)
print('VREF Write Response:', response1)
print('\n')

bmRequestType = 0b01000000
bRequest = 1
wValue = 0x0C
wIndex = 0
data = [0x01, 0x00, 0x00, 0x00]
data_hex = [hex(x) for x in data]
print('REG AUX Write data:', data_hex)
# Perform control transfer
response = dev.ctrl_transfer(bmRequestType, bRequest, wValue, wIndex, data)
print('REG AUX Write Bytes::', response)
print('\n')

## Channel REG
# Define control transfer parameters
bmRequestType = 0b01000000
bRequest = 1
wValue = 0x10
wIndex = 0
data = [0xFF, 0xFF]
data_hex = [hex(x) for x in data]
print('Channel Write data:', data_hex)
# Perform control transfer
response1 = dev.ctrl_transfer(bmRequestType, bRequest, wValue, wIndex, data)
print('Channel Write Response:', response1)
print('\n')

## START 
# Clear RESET & Start Capture
bmRequestType = 0b01000000
bRequest = 1
wValue = 0x04
wIndex = 0
data = [0x01, 0x00, 0x00, 0x00]
data_hex = [hex(x) for x in data]
print('REG CTRL Write data:', data_hex)
# Perform control transfer
response = dev.ctrl_transfer(bmRequestType, bRequest, wValue, wIndex, data)
print('REG CTRL Write Bytes:', response)
print('\n')

# Read data back from the device
## start_time = time.time()
## data_read_all = []
## while True:
##     data_read = dev.read(ep_in.bEndpointAddress, ep_in.wMaxPacketSize)
##     data_read_all.extend(data_read)
##     current_time = time.time()
##     if current_time - start_time >= 1:
##         break

start_time = time.time()
#Read 1MiB sample data
data_read = dev.read(ep_in.bEndpointAddress, 1*1024*1024) 
current_time = time.time()
data_read_all = data_read


## STOP
# Define control transfer parameters
bmRequestType = 0b01000000
bRequest = 1
wValue = 4
wIndex = 0
data = [0x00, 0x00, 0x00, 0x00]
data_hex = [hex(x) for x in data]
print('REG CTRL Write data:', data_hex)
# Perform control transfer
response1 = dev.ctrl_transfer(bmRequestType, bRequest, wValue, wIndex, data)
print('REG CTRL Write Response:', response1)

# Release the device resources
usb.util.dispose_resources(dev)

print('\n\n')

second = (current_time - start_time)
speed = ((len(data_read_all)/1024)/1024) / second

print(f'Finish Time {second}s')
print(f'USB Speed {speed} MiB/s')

print('\n\n')
for i in range(len(data_read_all)//1024):
    frame_start = [hex(x) for x in data_read_all[i*1024 : i*1024+16]]
    data_error = frame_start != ['0x0', '0x1', '0x2', '0x3', '0x4', '0x5', '0x6', '0x7', '0x8', '0x9', '0xa', '0xb', '0xc', '0xd', '0xe', '0xf']
    if data_error is True:
        print_start_color = "\033[91m"
    else:
        print_start_color = "\033[92m"
    print_end_color = "\033[0m"
    print(print_start_color + f'Data: {frame_start}' + print_end_color)




Device speed: Super Speed
    INTERFACE 0: Vendor Specific ===========================
     bLength            :    0x9 (9 bytes)
     bDescriptorType    :    0x4 Interface
     bInterfaceNumber   :    0x0
     bAlternateSetting  :    0x0
     bNumEndpoints      :    0x2
     bInterfaceClass    :   0xff Vendor Specific
     bInterfaceSubClass :   0xff
     bInterfaceProtocol :   0xff
     iInterface         :    0x0 
      ENDPOINT 0x2: Bulk OUT ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :    0x2 OUT
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :  0x400 (1024 bytes)
       bInterval        :    0x0
      ENDPOINT 0x82: Bulk IN ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :   0x82 IN
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :  0x400 (1024 bytes)
       bInter

USBError: [Errno 32] Pipe error